In [14]:
# dodajemo sve potrebne importe
import tensorflow as tf
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [15]:
# ucitavanje podataka
train, test = tfds.load( 'emnist/balanced', split=['train', 'test'], shuffle_files=True)
df_train = tfds.as_dataframe( train)
df_test = tfds.as_dataframe( test)

In [16]:
# inicijalno je dataset rotiran za 90 stupnjeva i zrcaljen, mozemo tranirati i na tome, ali mozemo ih i vratiti u originalan oblik
def reshape_rot_flip( img):
    img = np.rot90(img, k=-1, axes=(0,1))
    img = np.flip( img, (1))
    return img

In [17]:
# vracanje podataka u originalan oblik
x_train = np.stack( [reshape_rot_flip(img) for img in df_train['image']]) / 255
y_train = df_train['label'].to_numpy()
x_test = np.stack( [reshape_rot_flip(img) for img in df_test['image']]) / 255
y_test = df_test['label'].to_numpy()

In [18]:
# funkcija za crtanje
def plot_curve( epochs, hist, list_of_metrics):
    plt.figure()
    plt.xlabel( 'Epoch')
    plt.ylabel( 'Value')
    
    for m in list_of_metrics:
        x = hist[m]
        plt.plot( epochs[1:], x[1:], label=m)
        
    plt.legend()

In [19]:
# definiranje arhitekture modela
def create_model( learning_rate):
    model = tf.keras.models.Sequential()
    
    model.add( tf.keras.layers.Conv2D( filters=128, kernel_size=(3,3), activation='elu', input_shape=(28,28,1)))
    model.add( tf.keras.layers.MaxPool2D( pool_size=(2,2), strides=2))
    model.add( tf.keras.layers.Conv2D( filters=256, kernel_size=(3,3), activation='elu', padding='same'))
    model.add( tf.keras.layers.MaxPool2D( pool_size=(2,2), strides=2))
    model.add( tf.keras.layers.Dropout( rate=0.4))
    model.add( tf.keras.layers.Conv2D( filters=256, kernel_size=(3,3), activation='elu', padding='same'))
    model.add( tf.keras.layers.MaxPool2D( pool_size=(2,2), strides=2))
    
    model.add( tf.keras.layers.Flatten())
    model.add( tf.keras.layers.Dropout( rate=0.6))
    model.add( tf.keras.layers.Dense( 128, activation='elu'))
    model.add( tf.keras.layers.Dense( 47, activation='softmax'))
    
    model.compile( optimizer=tf.keras.optimizers.Adam( learning_rate=learning_rate),
                    loss="sparse_categorical_crossentropy",
                    metrics=['accuracy'])
    
    return model

# funkcija koja trenira model
def train_model( model, train_features, train_label, epochs,
                    batch_size=None, validation_split=0.1):

    history = model.fit( x=train_features, y=train_label, batch_size=batch_size,
                           epochs=epochs, shuffle=True,
                           validation_split=validation_split)
    
    epochs = history.epoch
    hist = pd.DataFrame( history.history)
    
    return epochs, hist

In [ ]:

# inicijalizacija nekih hyperparametara
learning_rate = 0.001
epochs = 500
batch_size = 2048
validation_split = 0.2

# stvori model
my_model = create_model( learning_rate)
  
# pokreni treniranje
epochs, hist = train_model( my_model, x_train, y_train,
                                  epochs, batch_size, validation_split)

# nacrtaj graf preciznosti po epohama
list_of_metrics_to_plot = ['accuracy']
plot_curve( epochs, hist, list_of_metrics_to_plot)

Epoch 1/500
45/45 [==============================] - 40s 816ms/step - loss: 1.7435 - accuracy: 0.5159 - val_loss: 0.6504 - val_accuracy: 0.7960
Epoch 2/500
45/45 [==============================] - 36s 803ms/step - loss: 0.6614 - accuracy: 0.7869 - val_loss: 0.5042 - val_accuracy: 0.8335
Epoch 3/500
45/45 [==============================] - 36s 804ms/step - loss: 0.5414 - accuracy: 0.8217 - val_loss: 0.4309 - val_accuracy: 0.8532
Epoch 4/500
45/45 [==============================] - 36s 805ms/step - loss: 0.4813 - accuracy: 0.8370 - val_loss: 0.4150 - val_accuracy: 0.8587
Epoch 5/500
45/45 [==============================] - 36s 807ms/step - loss: 0.4510 - accuracy: 0.8459 - val_loss: 0.3847 - val_accuracy: 0.8680
Epoch 6/500
45/45 [==============================] - 36s 807ms/step - loss: 0.4247 - accuracy: 0.8548 - val_loss: 0.3622 - val_accuracy: 0.8757
Epoch 7/500
45/45 [==============================] - 36s 808ms/step - loss: 0.4028 - accuracy: 0.8591 - val_loss: 0.3552 - val_accuracy:

In [11]:
my_model.metrics_names

['loss', 'accuracy']

In [12]:
my_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 6, 64)          36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 64)         0

In [13]:
my_model.evaluate( x=x_test, y=y_test, batch_size=batch_size)

10/10 [==============================] - 1s 52ms/step - loss: 0.3427 - accuracy: 0.8946


[0.34267792105674744, 0.8945744633674622]